<a href="https://colab.research.google.com/github/arshiatehrani/MLP-Hardcode/blob/PouyaSharifi/hoosh_s03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
import math
import numpy as np





In [ ]:
lst1 = [[1,2]]
lst2 = [[2],[3]]
lst3=[[2,3,5,6],[3,4,8,9]]
print(np.asarray(lst1) * np.asarray(lst3).T)
print(lst3,lst1)
print(np.asarray(lst3).T)

[[ 2  6]
 [ 3  8]
 [ 5 16]
 [ 6 18]]
[[2, 3, 5, 6], [3, 4, 8, 9]] [[1, 2]]
[[2 3]
 [3 4]
 [5 8]
 [6 9]]


In [ ]:
class MLP():
    def __init__(self,X,Y,X_val,Y_val,N_l,lr,L=1):
        self.X = np.concatenate((X,-1*(np.ones((X.shape[0],1)))),axis=1)
        self.X=self.X.T
        print(self.X)
        self.Y = Y
        print(self.Y)
        self.lr=lr
        # self.X_val = np.concatenate((X_val,np.ones((X_val.shape[0],1))),axis=1)
        # self.Y_val = np.squeeze(np.eye(10)[Y_val.astype(np.int).reshape(-1)])
        self.L = L
        self.N_l = N_l
        # self.n_samples = self.X.shape[0]
        self.layer_sizes =np.array([self.X.T.shape[1]]+[N_l]*L+[self.Y.T.shape[1]]) 
        print("[self.Y.shape[1]])" ,[self.Y.shape[1]]) 
        print("size",self.layer_sizes)
        self.__init_weights()
        # delete this portation if its not neccessary
        # self.train_loss = list()
        # self.train_acc = list()
        # self.val_loss = list()
        # self.val_acc = list()
        # self.train_time = list()
        # self.tot_time = list()
        # self.metrics = [self.train_loss,self.train_acc,self.val_loss,self.val_acc,self.train_time,self.tot_time]
    
    def __sigmoid(self,x):
        # VCompute the sigmoid
        return 1./(1.+np.exp(-x))
    
    def __softmax(self,x):
        # Compute softmax along the rows of the input
        exponent = np.exp(x)
        # print("softmax",x,exponent)
        # print("return esh axis 0",exponent.sum(axis=0,keepdims=True))
        # print(exponent/exponent.sum(axis=0,keepdims=True))
        # print("return esh axis 1",exponent.sum(axis=1,keepdims=True))
        # print(exponent/exponent.sum(axis=1,keepdims=True))
        
        # return exponent/exponent.sum(axis=1,keepdims=True)
        return exponent/exponent.sum(axis=0,keepdims=True)

    def __sigmoid_deriv(self,h):
        # Compute the derivative of sigmoid where h=sigmoid(x)
        return h*(1-h)
    def __init_weights(self):
        # Initialize the weights of the network given the sizes of the layers
        self.weights = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.weights.append(np.random.uniform(-1,1,size=[self.layer_sizes[i],self.layer_sizes[i+1]]))
        self.weights = np.asarray(self.weights)
    def init_layers(self,batch_size):
        # Initialize and allocate arrays for the hidden layer activations 
        self.__h = [np.empty((batch_size,layer)) for layer in self.layer_sizes]
    
    def feed_forward(self,batch):
        # Perform a forward pass of `batch` samples (N_samples x N_features)
        h_l = batch
        self.__h[0] = h_l
        for i,weights in enumerate(self.weights):
            # print("weights in this feed forward",self.weights)
            # print("h_l",h_l)
            # print("resualt zarb",h_l.dot(weights))
            h_l = self.__sigmoid(h_l.dot(weights))
            self.__h[i+1]=h_l
            
        self.out = self.__softmax(self.__h[-1])
        print("output",self.out)
    
    def back_prop(self,batch_y):
        # Update the weights of the network through back-propagation
        delta_t = (self.out - batch_y)*self.__sigmoid_deriv(self.__h[-1])
        for i in range(1,len(self.weights)+1):
            self.weights[-i]-=self.lr*(self.__h[-i-1].T.dot(delta_t))/self.batch_size
            delta_t = self.__sigmoid_deriv(self.__h[-i-1])*(delta_t.dot(self.weights[-i].T))
    def loss(self,y_pred,y):
        # Compute the loss along the rows, averaging along the number of samples
        return ((-np.log(y_pred))*y).sum(axis=1).mean()

    
    

In [ ]:
X_data_t=[[0,0,0],[0,0,1],[0,1,0],[0,1,1],[1,0,0],[1,0,1],[1,1,0],[1,1,1]]
Y_data=[[0,1,1,0,1,0,0,1]]
X_train=[[0,0,0],[0,0,1],[0,1,0],[0,1,1],[1,0,0],[1,0,1],[1,1,0]]
Y_train=[[0,1,1,0,1,0,0]]
X_test=[[1,1,1]]
Y_test=[[0]]
X_data=np.transpose(np.array(X_train))
print(x_data)
model = MLP(np.array(X_train),np.array(Y_train),X_test,Y_test,L=1,N_l=3,lr=1)
train_loss = 0
train_acc = 0
for epoch in range(4):
    lr=1.0
    n_samples = model.X.T.shape[0] 
    model.batch_size=2
    model.init_layers(model.batch_size)
    # model.__init_layers(model.batch_size)
    shuffle = np.random.permutation(n_samples)
    print(shuffle)
    X_batches = np.array_split(((model.X).T)[shuffle],n_samples/model.batch_size)
    Y_batches = np.array_split(((model.Y).T)[shuffle],n_samples/model.batch_size)
    print(X_batches,Y_batches)
    
    for batch_x,batch_y in zip(X_batches,Y_batches):
        print("batch x",batch_x)
        model.feed_forward(batch_x)  
        train_loss += model.loss(model.out,batch_y)
        # train_acc += model.__accuracy(model.__to_categorical(model.__out),batch_y)
        model.back_prop(batch_y)
        print("train_loss",train_loss)
batch_test=np.array([[0,0,0,-1]])
print("test_batch%%%",batch_test)
model.feed_forward(batch_test)


[[0 0 1 1]
 [0 1 0 1]
 [1 1 1 1]]
[[ 0.  0.  0.  0.  1.  1.  1.]
 [ 0.  0.  1.  1.  0.  0.  1.]
 [ 0.  1.  0.  1.  0.  1.  0.]
 [-1. -1. -1. -1. -1. -1. -1.]]
[[0 1 1 0 1 0 0]]
[self.Y.shape[1]]) [7]
size [4 3 1]
[0 1 2 6 3 5 4]
[array([[ 0.,  0.,  0., -1.],
       [ 0.,  0.,  1., -1.],
       [ 0.,  1.,  0., -1.]]), array([[ 1.,  1.,  0., -1.],
       [ 0.,  1.,  1., -1.]]), array([[ 1.,  0.,  1., -1.],
       [ 1.,  0.,  0., -1.]])] [array([[0],
       [1],
       [1]]), array([[0],
       [0]]), array([[0],
       [1]])]
batch x [[ 0.  0.  0. -1.]
 [ 0.  0.  1. -1.]
 [ 0.  1.  0. -1.]]
output [[0.33333371]
 [0.32372786]
 [0.34293843]]
train_loss 0.7326854715909716
batch x [[ 1.  1.  0. -1.]
 [ 0.  1.  1. -1.]]
output [[0.51515859]
 [0.48484141]]
train_loss 0.7326854715909716
batch x [[ 1.  0.  1. -1.]
 [ 1.  0.  0. -1.]]
output [[0.49223373]
 [0.50776627]]
train_loss 1.0715524888192813
[3 6 4 2 1 5 0]
[array([[ 0.,  1.,  1., -1.],
       [ 1.,  1.,  0., -1.],
       [ 1.,  0.,  0., 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
